In [11]:
import pandas as pd
import numpy as np
import json

In [12]:
filepath = '../shopping_lists/shoppingList1.json'

In [13]:
with open(filepath) as f:
    shopping_list = json.load(f)

In [183]:
def get_items(shopping_list):
    items_df = pd.DataFrame([i[0] for i in pd.DataFrame(shopping_list['aisles'])['items']])
    qties = pd.DataFrame(list(items_df['measures']))['metric']
    items_df.drop(['measures', 'usages', 'pantryItem', 'cost'], axis=1, inplace=True)
    items_df['amount'] = [i['amount'] for i in qties]
    items_df['unit'] = [i['unit'] for i in qties]
    items_df['name'] = items_df['name'].map(str.split)
    return items_df

In [187]:
items_df = get_items(shopping_list)

In [188]:
items_df

,id,name,aisle,ingredientId,amount,unit
0,589081,"[apple, cider, vinegar]","Oil, Vinegar, Salad Dressing",2048,59.8,ml
1,589083,"[vanilla, extract]",Baking,2050,0.4,ml
2,589091,[asparagus],Produce,11011,100.0,g
3,589133,"[vegan, cheese]",Cheese,93701,18.7,ml
4,589135,"[light-brown, sugar]",Pantry Items,19334,6.2,g
5,589161,"[milk, chocolate]",Sweet Snacks,19081,8.6,g
6,589163,"[cooked, black, beans]",Pasta and Rice,16015,28.7,g
7,589169,[toast],Bakery/Bread,18070,2.0,pieces
8,589173,"[salted, butter]","Milk, Eggs, Other Dairy",1001001,14.2,g
9,589181,"[chicken, broth]",Canned and Jarred,6194,29.4,ml


In [20]:
import requests

In [220]:
r = requests.get('http://localhost:8000/products?format=json')

In [221]:
shop_df = pd.DataFrame(json.loads(r.content.decode()))

In [222]:
raw = shop_df.copy()

In [223]:
shop_df['name'] = [i.lower() for i in shop_df['name']]

In [224]:
shop_df.head()

,store,name,img_url,price,currency,amount,unit
0,sainsburys,sainsbury's fairtrade bananas loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,73.0,p,Loose,kg
1,sainsburys,sainsbury's carrots loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,41.0,p,Loose,kg
2,sainsburys,sainsbury's whole cucumber,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,43.0,p,,unit
3,sainsburys,sainsbury's onions loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Sa...,85.0,p,Loose,kg
4,sainsburys,sainsbury's baking potatoes loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,64.0,p,Loose,kg


In [225]:
product = items_df.iloc[0]

In [226]:
product

id                                    589081
name                 [apple, cider, vinegar]
aisle           Oil, Vinegar, Salad Dressing
ingredientId                            2048
amount                                  59.8
unit                                      ml
Name: 0, dtype: object

In [243]:
shop_df[shop_df['name'].str.contains('(?=.*black)(?=.*beans)')]

,store,name,img_url,price,currency,amount,unit
745,sainsburys,"sainsbury's black beans carton, so organic 380...",https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,75.0,p,(230g*),unit


In [286]:
matches_df = pd.DataFrame()
for product in items_df.iloc:
    regex = ''.join([f'(?=.*{i})' for i in product['name']])   
    entries = shop_df['name'].str.contains(regex)    
    if entries.any():
        matches = shop_df[entries]
        print(f"appending {' '.join(product['name'])}")
        matches_df = matches_df.append(matches)
    else:
        print(f"no such product: {' '.join(product['name'])}")

no such product: apple cider vinegar
no such product: vanilla extract
no such product: asparagus
appending vegan cheese
no such product: light-brown sugar
appending milk chocolate
no such product: cooked black beans
appending toast
appending salted butter
appending chicken broth
no such product: red chile flakes
no such product: agave
appending sirloin steak
no such product: edamame
no such product: tofu
no such product: pecans


In [287]:
matches_df.sample(frac=0.2)

,store,name,img_url,price,currency,amount,unit
847,sainsburys,kitkat 4 finger milk chocolate bar multipack 4...,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.00,GBP,4x41.5g,unit
720,sainsburys,cadbury dairy milk chocolate bar 200g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.50,GBP,200g,unit
723,sainsburys,cadbury dairy milk giant chocolate buttons bag...,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.50,GBP,119g,unit
901,sainsburys,cadbury dairy milk chocolate bar 10x29.3g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,2.00,GBP,10x29.3g,unit
1089,sainsburys,"sainsbury's indulgence ice cream, milk chocola...",https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.35,GBP,4x110ml,unit
702,sainsburys,mcvitie's milk chocolate digestives biscuits 266g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.60,GBP,266g,unit
632,sainsburys,sainsbury's toastie thick sliced white bread 800g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,55.00,p,800g,unit


okay so for now main problem is that there are missing products in our database, exotic stuff which for some reason is in this shopping list

another issue is that sometimes there is like light-brown sugar which would be brown sugar basically, or red chile flakes instead of chilli flakes, edamame instead of edame so some food dict and matching shit could probably be added too

also Im not really sure why its the case here that we cant get a shopping list per meal plan and its one per account, Ill check out if it has to do with bind user soon, this would be a nice option we could have a user per meal plan then. I would need a few more shopping lists to create a proper parsing program which will still be a pretty complex task

I will have to use a wide logic and the more ingredients - the better

I think that making a user account for now is not an issue, as a matter of fact I can implement async storage so that the state is going to be preserved after app restart

additional thing to remember here is that we have to track the amount of ingredients each user has, or for now dont worry about re-suggesting to buy butter each week if a user hasnt used up all the butter yet